In [6]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict
from dotenv import load_dotenv

In [7]:
load_dotenv()

True

In [8]:
model = ChatGoogleGenerativeAI(model='gemini-2.5-flash')

In [9]:
# creat state

class LLMState(TypedDict):

    question: str
    answer: str

In [10]:
def llm_qa(state: LLMState) -> LLMState:

    # extract the question from the state
    question = state['question']

    # from a prompt
    prompt = f'Answer the following question: {question}'

    # ask that question to the LLM
    answer = model.invoke(prompt).content

    # update the answer in the state
    state['answer'] = answer

    return state

In [12]:
# create our graph
graph = StateGraph(LLMState)

# add nodes
graph.add_node('llm_qa', llm_qa)

# add edges
graph.add_edge(START,'llm_qa')
graph.add_edge('llm_qa', END)

# compile the graph
workflow = graph.compile()

In [13]:
#execute

initial_state = {'question': 'How far is moon from the earth?'}

final_state = workflow.invoke(initial_state)

print(final_state)

{'question': 'How far is moon from the earth?', 'answer': "The distance between the Earth and the Moon is not constant because the Moon's orbit around Earth is elliptical, not a perfect circle.\n\nHere are the key distances:\n\n*   **Average distance:** Approximately **384,400 kilometers (238,900 miles)**\n*   **Perigee (closest point):** Approximately **363,104 kilometers (225,623 miles)**\n*   **Apogee (farthest point):** Approximately **405,696 kilometers (252,088 miles)**\n\nSo, while the average is often cited, the actual distance varies throughout the lunar cycle."}


In [14]:
model.invoke('how far is moon from the earth?').content

"The distance between the Earth and the Moon is not constant, as the Moon orbits the Earth in an elliptical path.\n\nHowever, the **average distance** is approximately:\n\n*   **384,400 kilometers (km)**\n*   **238,900 miles**\n\nHere's a breakdown of the range:\n\n*   **At its closest (perigee):** About **363,104 km (225,623 miles)**\n*   **At its farthest (apogee):** About **405,696 km (252,088 miles)**\n\nTo give you a sense of scale, you could fit about 30 Earths between our planet and the Moon. Also, light takes about 1.28 seconds to travel that average distance."